<a href="https://colab.research.google.com/github/Ranya-Alghamdi-1/T5/blob/main/YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install opencv-python-headless ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import drive
from IPython.display import Video

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_path = '/content/drive/MyDrive/T5/Week 6/5- Weekly Project (Sunday)/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt'
model = YOLO(model_path)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
video_path = '/content/drive/MyDrive/T5/Week 6/5- Weekly Project (Sunday)/5- Weekly Project (Sunday)/Dataset/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output_path = '/content/output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
import cv2
import numpy as np

In [ ]:
#Defining the Regions of Interest (ROIs) for detecting vehicles
ROI_top = (50, 50, 300, 150)
ROI_bottom = (50, 400, 300, 500)
ROI_left = (0, 200, 100, 400)
ROI_right = (500, 200, 600, 400) #Random ROIs

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
count_top = 0
count_bottom = 0
count_left = 0
count_right = 0

In [ ]:
# Function to check if a vehicle's center is within ROI
def is_within_roi(center, roi):
    x1, y1, x2, y2 = roi
    return x1 < center[0] < x2 and y1 < center[1] < y2

# Process Video Frames and Identify Vehicles
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results[0].boxes:
        box = result.xyxy[0]
        x1, y1, x2, y2 = map(int, box)
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        label = result.cls
        confidence = result.conf
        color = (0, 255, 0)


0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2935.0ms
Speed: 28.8ms preprocess, 2935.0ms inference, 31.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 3832.4ms
Speed: 4.1ms preprocess, 3832.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 3279.9ms
Speed: 7.1ms preprocess, 3279.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2671.1ms
Speed: 3.8ms preprocess, 2671.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2684.5ms
Speed: 3.9ms preprocess, 2684.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2704.5ms
Speed: 3.9ms preprocess, 2704.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 4312.8ms
Speed: 3.8ms

In [ ]:
# Draw bounding box, center
cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
cv2.circle(frame, (center_x, center_y), 5, color, -1)
cv2.putText(frame, f'{label} {int(confidence * 100)}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Check if the vehicle exited the road of the ROIs
if is_within_roi((center_x, center_y), ROI_top):
            count_top += 1
elif is_within_roi((center_x, center_y), ROI_bottom):
            count_bottom += 1
elif is_within_roi((center_x, center_y), ROI_left):
            count_left += 1
elif is_within_roi((center_x, center_y), ROI_right):
            count_right += 1

# Draw the ROIs on the frame
cv2.rectangle(frame, ROI_top[:2], ROI_top[2:], (255, 0, 0), 2)
cv2.rectangle(frame, ROI_bottom[:2], ROI_bottom[2:], (255, 0, 0), 2)
cv2.rectangle(frame, ROI_left[:2], ROI_left[2:], (255, 0, 0), 2)
cv2.rectangle(frame, ROI_right[:2], ROI_right[2:], (255, 0, 0), 2)

# Display the vehicle counts
cv2.putText(frame, f'Top : {count_top}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(frame, f'Bottom : {count_bottom}', (50, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(frame, f'Left : {count_left}', (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(frame, f'Right : {count_right}', (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
**I searched in google and github**


In [ ]:
# Display my name on the top
cv2.putText(frame, "Raniyah Alghamdi", (width - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

## Save and Submit Your Work

In [ ]:
out.write(frame)
# Release resources
cap.release()
out.release()

In [ ]:
# Convert the video to mp4 format using ffmpeg
!ffmpeg -y -loglevel panic -i /content/output_video.avi /content/final_video.mp4

# Display the final video
Video('/content/final_video.mp4', embed=True, width=960)

Submit both the notebook and the output video